In [1]:
import os
import requests
import json

In [23]:
WB_TOKEN = os.getenv('WB_TOKEN')


def upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1):
  
    url = "https://content-api.wildberries.ru/content/v3/media/file"
    
    headers = {
        "Authorization": api_key,
        "X-Nm-Id": str(nm_id),
        "X-Photo-Number": str(photo_number)
    }
    
    with open(file_path, "rb") as f:
        files = {
            "uploadfile": (os.path.basename(file_path), f, "image/jpeg")
        }
        
        try:
            response = requests.post(url, headers=headers, files=files)
        except requests.RequestException as e:
            print(f"Ошибка при отправке запроса: {e}")
            return
        
        if response.status_code == 200:
            resp_json = response.json()
            if not resp_json.get("error", False):
                print(f"Успешно загружено изображение '{os.path.basename(file_path)}' на nmID={nm_id}.")
            else:
                print(f"Ошибка Wildberries при загрузке: {resp_json.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера. Код: {response.status_code}, файл: {os.path.basename(file_path)}")

def process_vendor_list(vendor_list, images_folder, api_key):
    """
    Проходит по списку vendor_list, проверяет, существует ли соответствующий файл в images_folder,
    и при наличии вызывает функцию загрузки на WB.
    
    :param vendor_list: Список словарей вида [{'nmID': ..., 'vendorCode': ' zip_lock_XXXX_YY'}, ...]
    :param images_folder: Путь к папке, где лежат файлы (например, 'images_900_1200').
    :param api_key: API-ключ для Wildberries.
    """
    for item in vendor_list:
        nm_id = item["nmID"]
        vendor_code = item["vendorCode"]
        
        # Ожидаем формат vendorCode = "zip_lock_9604677_30"
        # Удаляем " zip_lock_" в начале:
        # Неподходящий формат vendorCode: 'zip_lock_9604896_100'. Пропускаем.
        
        if vendor_code.startswith("zip_lock_"):
        
            code_part = vendor_code[8:]  # "9604677_30"
            # Разделяем по символу "_"
            parts = code_part.split("_", maxsplit=1)
            if len(parts) == 2:
                product_id, quantity_str = parts
                # Формируем имя файла: "9604677-30.jpg"
                image_name = f"{product_id}-{quantity_str}.jpg"
                file_path = os.path.join(images_folder, image_name)
                
                # Проверяем, существует ли файл
                if os.path.isfile(file_path):
                    print(f"Файл '{image_name}' найден. Загружаем на nmID={nm_id}...")
                    upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1)
                else:
                    print(f"Файл '{image_name}' не найден в папке '{images_folder}'. Пропускаем.")
            else:
                print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")
        else:
            print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")


In [3]:
all_cards_list = "https://content-api.wildberries.ru/content/v2/get/cards/list"
headers = {
    "Authorization": WB_TOKEN,
}
body =             {
          "settings": {                      
            "cursor": {
              "limit": 100
            },
            "filter": {
              "withPhoto": 0
            }
          }
        }


resp = requests.post(all_cards_list, headers=headers, json=body)

In [20]:
d = json.loads(resp.text)
d

{'cards': [{'nmID': 324345974,
   'imtID': 305703298,
   'nmUUID': '0194c36b-089a-7f95-b58d-68464d833b51',
   'subjectID': 3756,
   'subjectName': 'Пакеты упаковочные',
   'vendorCode': 'zip_lock_9604896_100',
   'brand': '',
   'title': 'Пакеты Zip Lock, 200*300 мм, 70 мкм, 100 шт.',
   'description': 'Пакеты Zip-Lock, прозрачного цвета с красной полосой, длиной 300 мм, шириной 200 мм, толщиной 70 мкм, в упаковке по 100 штук. Материал – полиэтилен высокого давления (ПВД). Производятся в России.\nОбласть применения:\nПакеты Зип-Лок применяются для упаковки продукции небольшого размера. Прозрачный пакет позволяет разглядеть содержимое, застежка zip-lock герметично закрывает пакет, обеспечивает сохранность продукта и возможность удобной транспортировки различных веществ. Используется при упаковке сыпучих товаров: чая, кофе, орехов, круп и мелких деталей.\nПреимущества:\nПрезентабельный внешний вид;\nПрочность;\nПростота и герметичность упаковки;\nНизкая цена.',
   'needKiz': False,
   'd

In [21]:
newNmIdVendorCodes = [{"nmID": row["nmID"], "vendorCode": row["vendorCode"]} for row in d["cards"]]

In [11]:
newNmIdVendorCodes

[{'nmID': 324345974, 'vendorCode': 'zip_lock_9604896_100'},
 {'nmID': 324345976, 'vendorCode': 'zip_lock_9603211_100'},
 {'nmID': 324345978, 'vendorCode': 'zip_lock_9604894_100'},
 {'nmID': 324345982, 'vendorCode': 'zip_lock_9604895_100'},
 {'nmID': 324345975, 'vendorCode': 'zip_lock_9700114_100'},
 {'nmID': 324345972, 'vendorCode': 'zip_lock_9603149_100'},
 {'nmID': 324345980, 'vendorCode': 'zip_lock_9604893_100'},
 {'nmID': 324345979, 'vendorCode': 'zip_lock_9603148_100'},
 {'nmID': 324345981, 'vendorCode': 'zip_lock_9700115_100'},
 {'nmID': 324345970, 'vendorCode': 'zip_lock_9603210_100'},
 {'nmID': 324345977, 'vendorCode': 'zip_lock_9604897_100'},
 {'nmID': 324345984, 'vendorCode': 'zip_lock_9700117_100'},
 {'nmID': 324345983, 'vendorCode': 'zip_lock_9604892_100'},
 {'nmID': 324345985, 'vendorCode': 'zip_lock_9700116_100'},
 {'nmID': 324345971, 'vendorCode': 'zip_lock_9602833_100'},
 {'nmID': 324345969, 'vendorCode': 'zip_lock_9604898_100'},
 {'nmID': 324346455, 'vendorCode': 'zip_

In [24]:
process_vendor_list(newNmIdVendorCodes, "./zip_locks_images_900_1200", WB_TOKEN)

1 zip_lock_9604896_100
Неподходящий формат vendorCode: 'zip_lock_9604896_100'. Пропускаем.
1 zip_lock_9603211_100
Неподходящий формат vendorCode: 'zip_lock_9603211_100'. Пропускаем.
1 zip_lock_9604894_100
Неподходящий формат vendorCode: 'zip_lock_9604894_100'. Пропускаем.
1 zip_lock_9604895_100
Неподходящий формат vendorCode: 'zip_lock_9604895_100'. Пропускаем.
1 zip_lock_9700114_100
Неподходящий формат vendorCode: 'zip_lock_9700114_100'. Пропускаем.
1 zip_lock_9603149_100
Неподходящий формат vendorCode: 'zip_lock_9603149_100'. Пропускаем.
1 zip_lock_9604893_100
Неподходящий формат vendorCode: 'zip_lock_9604893_100'. Пропускаем.
1 zip_lock_9603148_100
Неподходящий формат vendorCode: 'zip_lock_9603148_100'. Пропускаем.
1 zip_lock_9700115_100
Неподходящий формат vendorCode: 'zip_lock_9700115_100'. Пропускаем.
1 zip_lock_9603210_100
Неподходящий формат vendorCode: 'zip_lock_9603210_100'. Пропускаем.
1 zip_lock_9604897_100
Неподходящий формат vendorCode: 'zip_lock_9604897_100'. Пропускаем.